## Lab Assignment 2
### Data Mining 7331 Section 403
---
- Brian Coari
- Stephen Merritt
- Cory Thigpen
- Quentin Thomas

This dataset tracks all Olympians' medal results from 1896-2016, as well as various physical attributes such as `Sex`, `Age`, `Weight`, and `Height`. In this lab we will (1) predict podium finishers using binary classification and (2) predict the sport a podium-finishing athlete played using multinomial classification.

### Data Preparation

#### Section 1

- (10pts) Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.

---

In [11]:
import numpy as np
import pandas as pd
df = pd.read_csv('~/olympics/data/athletes_cleaned_merged.csv')
df.head()

,Unnamed: 0,NOC,Country,ID,Name,Sex,Age,Height,Weight,Team,Games,Year,Season,City,Sport,Event,Medal
0,872,AFG,Afghanistan,502.0,Ahmad Shah Abouwi,M,25,176,73,Afghanistan,1956 Summer,1956,Summer,Melbourne,Hockey,Hockey,No Medal
1,1950,AFG,Afghanistan,1076.0,Jammal-ud-Din Affendi,M,28,176,73,Afghanistan,1936 Summer,1936,Summer,Berlin,Hockey,Hockey,No Medal
2,1992,AFG,Afghanistan,1101.0,Mohammad Anwar Afzal,M,23,177,73,Afghanistan,1948 Summer,1948,Summer,London,Football,Football,No Medal
3,3129,AFG,Afghanistan,1745.0,Mohammad Aktar,M,17,156,48,Afghanistan,1980 Summer,1980,Summer,Moskva,Wrestling,"Light-Flyweight, Freestyle",No Medal
4,8410,AFG,Afghanistan,4628.0,Mohammad Daoud Anwary,M,22,172,76,Afghanistan,1964 Summer,1964,Summer,Tokyo,Wrestling,"Bantamweight, Freestyle",No Medal


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266954 entries, 0 to 266953
Data columns (total 17 columns):
Unnamed: 0    266954 non-null int64
NOC           266954 non-null object
Country       266954 non-null object
ID            266954 non-null float64
Name          266954 non-null object
Sex           266954 non-null object
Age           266954 non-null int64
Height        266954 non-null int64
Weight        266954 non-null int64
Team          266954 non-null object
Games         266954 non-null object
Year          266954 non-null int64
Season        266954 non-null object
City          266954 non-null object
Sport         266954 non-null object
Event         266954 non-null object
Medal         266954 non-null object
dtypes: float64(1), int64(5), object(11)
memory usage: 34.6+ MB


The intial data set contained the following data of different types:

| Nominal | Ordinal | Numeric | Binary |
| ------- | ------- | ------- | ------ |
| Games   | City    | Age     |        |
| Name    | Event   | Height  |        |
| NOC     | ID      | Weight  |        |
| Season  | Medal   | Year    |        |
| Sex     |         |         |        |
| Team    |         |         |        |

To prepare the lab, we brought population data into the data set and changed all National Olympic Committee or `NOC` data points to match their corresponding `Country`. For countries without entries in the worldbank URL we hard-coded the populations. There were 106 observations in the data set not associated with a country, so we dropped them. These were `Refugee Athletes` and `Individual Olympic Athletes`. `Sex`, `Season`, and `Medal` were converted to numeric factors and `ID`, `NOC`, `Name`, `Team`, and `Games` were dropped due to lack of relevancy.

In [13]:
df_pop = pd.read_csv('~/olympics/data/POP_TOTAL.csv', encoding = "ISO-8859-1")
df_pop = df_pop[['Country Code', '2015']]
#Change all Russia NOCs
df['NOC'] = np.where(df['NOC'] == 'EUN', 'RUS', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'URS', 'RUS', df['NOC'])
#Change all Australia NOCs
df['NOC'] = np.where(df['NOC'] == 'ANZ', 'AUS', df['NOC'])
#Change all German NOCs
df['NOC'] = np.where(df['NOC'] == 'FRG', 'GER', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'GDR', 'GER', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'SAA', 'GER', df['NOC'])
#Change all Congo NOCs
df['NOC'] = np.where(df['NOC'] == 'CGO', 'COD', df['NOC'])
#Change all Czech NOCs
df['NOC'] = np.where(df['NOC'] == 'BOH', 'CZE', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'TCH', 'CZE', df['NOC'])
#Change all Yemen NOCs
df['NOC'] = np.where(df['NOC'] == 'YAR', 'YEM', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'YMD', 'YEM', df['NOC'])
#Change all Greek NOCs
df['NOC'] = np.where(df['NOC'] == 'CRT', 'GRE', df['NOC'])
#Change all Zimbabwe NOCs
df['NOC'] = np.where(df['NOC'] == 'RHO', 'ZIM', df['NOC'])
#Change all Malaysia NOCs
df['NOC'] = np.where(df['NOC'] == 'MAL', 'MAS', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'NBO', 'MAS', df['NOC'])
#Change all Vietnam NOCs
df['NOC'] = np.where(df['NOC'] == 'VNM', 'VIE', df['NOC'])
#Change all Trinidad and Tobego NOCs
df['NOC'] = np.where(df['NOC'] == 'WIF', 'TTO', df['NOC'])
#Change all Trinidad and Tobego NOCs
df['NOC'] = np.where(df['NOC'] == 'UAR', 'SYR', df['NOC'])
#Change all Serbian NOCs
df['NOC'] = np.where(df['NOC'] == 'SCG', 'YUG', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'SRB', 'YUG', df['NOC'])
#Change all Canada NOCs
df['NOC'] = np.where(df['NOC'] == 'NFL', 'CAN', df['NOC'])
df_pop.rename(index = str, columns = {'Country Code' : 'NOC', '2015': '2015 Population'}, inplace = True)
df_pop_merge = pd.merge(left = df, right = df_pop, how = 'left', on=['NOC'], left_index=False)
df_pop_merge.loc[df_pop_merge['Country'] == 'Taiwan', '2015 Population'] =  23485755
df_pop_merge.loc[df_pop_merge['Country'] == 'Saint Vincent', '2015 Population'] =  109643
df_pop_merge.loc[df_pop_merge['Country'] == 'Palestine', '2015 Population'] =  4817000
df_pop_merge.loc[df_pop_merge['Country'] == 'Cook Islands', '2015 Population'] =  17459
df_pop_merge.loc[df_pop_merge['Country'] == 'Eritrea', '2015 Population'] =  4846976
df_pop_merge.loc[df_pop_merge['Country'] == 'Saint Kitts', '2015 Population'] =  54821
df_pop_mergena = df_pop_merge[df_pop_merge['2015 Population'].isnull()]
#Drop the 106 observations not associated with a country 
df_pop_merge = df_pop_merge.dropna(how = 'any')

In [15]:
#Calculate the population proportion for each country
df_pop = df_pop_merge.groupby(['Country'])['2015 Population'].mean().reset_index()
df_pop['Population_Prop'] = df_pop['2015 Population'].transform(lambda x: x / (x.sum()))

#Calculate BMI for each athlete
def calculate_bmi(df):
    return (df[1]/(df[0] * df[0])) * 10000
df_pop_merge['BMI'] = df_pop_merge[['Height', 'Weight']].apply(calculate_bmi, axis=1)

#Merge dataframes with Population Propotion information
df_pop_merge = pd.merge(left = df_pop_merge, right = df_pop, how = 'left', on=['Country'], 
                        left_index=False)
#Create Binary Medal, Season, and Sex Classes
df_pop_merge['Medal'] = np.where(df_pop_merge.Medal == 'No Medal', 0, 1)
df_pop_merge['Season'] = np.where(df_pop_merge.Season == 'Summer', 1, 0)
df_pop_merge['Sex'] = np.where(df_pop_merge.Sex == 'M', 1, 0)

In [16]:
%%time

prev_medals = df_pop_merge.groupby(['Name', 'Year'])['Medal'].sum()
prev_medals = prev_medals.reset_index()
prev_medals.columns = ['Name', 'Year', 'Medal_Count']
prev_medals['Previous_Medals'] = prev_medals['Medal_Count']

for i in range(2, len(prev_medals)):
    if prev_medals.iloc[i]['Name'] == prev_medals.iloc[i-1]['Name']:
        if prev_medals.iloc[i-1]['Medal_Count'] != 0:
            prev_medals['Previous_Medals'].iloc[i] = 1
        else:
            prev_medals['Previous_Medals'].iloc[i] = 0
    else:
        prev_medals['Previous_Medals'].iloc[i] = 0

/Users/Steve/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


CPU times: user 1h 38min 49s, sys: 12.6 s, total: 1h 39min 2s
Wall time: 1h 39min 11s


In [17]:
#Merge prev_medals into existing pop_merge dataset to add Previous_Medals feature
df_prev_medal_merge = pd.merge(left = df_pop_merge, right = prev_medals, how = 'left', on = ['Name', 'Year'])
df_pop_merge = df_prev_medal_merge[['Country', 'Sex', 'Age', 'Height', 'Weight', 'Year', 'Season', 'Sport', 'Event', 'BMI', 
                             'Population_Prop', 'Medal', 'Previous_Medals']]

#Write dataframe to .csv file in order to preserve the data and avoid Feature Engineering code in future iterations 
df_pop_merge.to_csv('~/olympics/data/pop_merge.csv')

In [18]:
#Create X and y vectors for sklearn pre-processing steps
df = pd.read_csv('~/olympics/data/pop_merge.csv')

y = df['Medal']
X = df.drop(['Medal', 'Unnamed: 0'], axis = 1)

print(X.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266848 entries, 0 to 266847
Data columns (total 12 columns):
Country            266848 non-null object
Sex                266848 non-null int64
Age                266848 non-null int64
Height             266848 non-null int64
Weight             266848 non-null int64
Year               266848 non-null int64
Season             266848 non-null int64
Sport              266848 non-null object
Event              266848 non-null object
BMI                266848 non-null float64
Population_Prop    266848 non-null float64
Previous_Medals    266848 non-null int64
dtypes: float64(2), int64(7), object(3)
memory usage: 24.4+ MB
None


`Country`, `Sport`, and `Event` had too many variables for binary coding, so one-hot encoding was performed.

In [19]:
X = pd.get_dummies(X, columns = ['Country', 'Event', 'Sport'])
print(X.describe())
print(y.describe())

                 Sex            Age         Height         Weight  \
count  266848.000000  266848.000000  266848.000000  266848.000000   
mean        0.722786      25.330338     175.420948      71.017542   
std         0.447624       5.801661       9.869066      13.330644   
min         0.000000      10.000000     127.000000      25.000000   
25%         0.000000      21.000000     169.000000      62.000000   
50%         1.000000      24.000000     176.000000      71.000000   
75%         1.000000      28.000000     182.000000      78.000000   
max         1.000000      72.000000     226.000000     214.000000   

                Year         Season            BMI  Population_Prop  \
count  266848.000000  266848.000000  266848.000000    266848.000000   
mean     1979.016369       0.818185      22.904691         0.013419   
std        29.560519       0.385693       2.718862         0.028553   
min      1896.000000       0.000000       8.360954         0.000001   
25%      1960.000000   

In [20]:
from sklearn.preprocessing import StandardScaler
#Standardize X
sc = StandardScaler()
sc.fit(X)
X_std = sc.transform(X)

#### Section 2

- (5pts) Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).

---

Three variables were added to the data set: `Population_Prop`, `BMI`, and `Previous Medals`. The `Population_Prop` variable was added to represent the percentage of the overall world population for a given `Country`. Athlete `BMI` was added to the dataframe where $\omega$ represents `Weight` in kilograms, and $\eta$ represents `Height` in centimeters, and $C$ is a constant at `10,000`:

$$
BMI = \left(\frac{\omega}{\eta \eta}\right) C
$$

`Previous_Medals` was created as a binary representation if an athlete had won a medal at a previous Olympic games (indicated by a `1`). The `Medal` variable was converted from ordinal to binary, where `Gold`, `Silver`, or `Bronze` were given a `1` and `No Medal` performers were given a `0`. `Season` and `Sex` were also converted to binary. 

One-hot encoding was performed for binarization of the remaining nominal variables(`Country`, `Event`, and `Sport`) so they could be included as features to train the classification model.

The final data set contains the following data of different types:

| Nominal | Ordinal | Numeric         | Binary          |
| ------- | ------- | --------------- | --------------- | 
|         |         | Age             | Country         |
|         |         | BMI             | Event           |
|         |         | Height          | Medal           |
|         |         | Population_Prop | Previous-Medals |
|         |         | Weight          | Season          |
|         |         | Year            | Sex             |
|         |         |                 | Sport           |

### Modeling and Evaluation

#### Section 1

- (10 pts) Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.

---

#### Section 2

- (10pts) Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. For example, if you are using time series data then you should be using continuous training and testing sets across time.

---

In [ ]:
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=10, test_size=0.20, random_state=0)

#### Section 3

- (20pts) Create three different classification/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric. You must investigate different parameters of the algorithms!

---

In [ ]:
# Classifer Estimator created and edited from Professor Drew's NC school repository
def EvaluateClassifierEstimator2(classifierEstimator, X, y, cv):
    
    #Perform cross validation 
    from sklearn.model_selection import cross_val_predict
    predictions = cross_val_predict(classifierEstimator, X, y, cv=cv)
    
    #model evaluation 
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score 
    
    #pass true test set values and predictions to classification_report
    classReport = classification_report(y, predictions)
    confMat = confusion_matrix(y, predictions)
    acc = accuracy_score(y, predictions)

    
    print(classReport)
    print(confMat)
    print(acc)

In [ ]:
#Logisitic regression 10-fold cross-validation 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
lr = LogisticRegression()


parameters = { 'penalty':['l2','l1']
              ,'C': [0.001, 0.01, 0.1, 1, 10, 100]
              ,'class_weight': ['balanced','none']
              ,'random_state': [0]
              ,'solver': ['sag', 'saga']
              ,'max_iter':[100,500]
             }

#Create a grid search object using the  
regGS = GridSearchCV(estimator=lr
                   , n_jobs=-1 #Use all possible cores to run jobs in parallel
                   , verbose=True
                   , param_grid=parameters
                   , cv=cv # KFolds = 10
                   , scoring='f1')

#Perform hyperparameter search to find the best combination of parameters for our data
regGS.fit(X_std, Y)

In [ ]:
lr_clf = GS.best_estimator_

In [ ]:
EvaluateClassifierEstimator2(lr_clf, X_std, y, 10)

In [ ]:
lr_clf = lr_clf.fit(X_std, y)

In [ ]:
from sklearn.externals import joblib
gs_lr_model = 'gs_lr_model.joblib.pkl'
_ = joblib.dump(lr_clf, gs_lr_model, compress = 9)
#This will save the model in your working directory

In [ ]:
clf2 = joblib.load(gs_lr_model)
clf2

#### Section 4

- (10pts) Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model.

---

#### Section 5

- (10pts) Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation as discussed in unit 7 of the course.

---

#### Section 6

- (10pts) Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task.

---

### Deployment

- (5pts) How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.? 

---

These predictions could be used for predicting medal winners in future Olympics based on variables such as `BMI`, `Height`, `Weight`, or `Sex`; however, this is a bit impractical due to the lack of discernable difference in skillset. Individuals with similar characteristics do not necessarily perform the same. To make podium predictions more useful we would need a skill level metric, but skill levels change over time. Consequently, this data set is better served for informational or educational purposes.

To teach an individual or organization about <i>data science</i>, <i>machine learning</i>, or <i>data mining</i> we must first produce something relatable and <u>this</u> data set is something people understand, so as we explain what classifications we are making the content becomes understandable. From there we can dive into data that is more relavent to that individual or organization.

Binary and multinomial classification have numerous use cases across industries. For example:

| Industry      | Use Case                |
| ------------- | ----------------------- |
| Advertising   | Targeted advertising    |
| Energy        | Prediction of dry well  |
| Manufacturing | Risk of machine failure |
| Sports        | Run or pass in football |
| Telecom       | Customer retention      |
| Underwriting  | Risk level of borrower  |

To <b>measure the value</b> of any prediciton model we would develop a business case. To do this we would quantify risk reduction, revenue increases, changes in working capital, and/or cost decreases that were a direct result of the model. Next we would determine the cost of implementing the model and perform a discounted cash flow analysis to ensure there was a positive return on investment.

To <b>deploy</b> a model under any of the aforementioned use cases we would need to understand the architecture environment and determine how to use APIs to connect to the classification model. Once this is established the predictor could be integrated with an existing system or set up on a clean build.

It may take many hours to <b>train</b> a prediction model, so it would be impractical to retrain a model in real-time using a relational database. Instead, the frequency for updating the prediction model would need to be determined by the nature of the use case. Using the telecom example from above, customer retention prediction models would likely run monthly to align with billing and usage limits.

### Exceptional Work

- (10pts) You have free reign to provide additional analyses. One idea: grid search parameters in a parallelized fashion and visualize the performances across attributes. Which parameters are most significant for making a good model for each classification algorithm?

---